## Update example YAML resources

- read file from folder
- convert yaml to dict
- add element
    with data from file name
    optionally user input - future
- validate
- save file back to folder

In [31]:
from pathlib import Path
from json import dumps, loads
from fhir.resources.valueset import ValueSet
from fhir.resources.codesystem import CodeSystem
from requests import post
from yaml import load as y_load, dump as y_dump, FullLoader
from IPython.display import display, Markdown, HTML
from datetime import date

date.today().strftime("%Y-%m-%d")

'2022-10-24'

In [32]:
#in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/output')
in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/resources-yaml')
# in_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/output')
# out_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml')

In [34]:
prune= ['text','version','publisher','contact','jurisdiction']
for i in in_path.glob(f'codesystem-*.yml'):
# for i in in_path.glob('valueset-*.yml'):
    print(i)
    vs_obj = CodeSystem.parse_file(i)
    # vs_obj = ValueSet.parse_file(i)
    for p in prune:
        try:
            delattr(vs_obj,p)
        except AttributeError:
            pass
    vs_obj.experimental = False
    vs_obj.date = date.today()
    yml_str = vs_obj.yaml(indent=True, sort_keys=False)
    print( f"writing {i.stem} to {i}")
    # print(yml_str)
    i.write_text(yml_str)
    
    # for j in out_path.glob('ValueSet-*.yml'):
    #     # print(f'{i.stem} == {j.stem} = {i.stem == j.stem}')
    #     if i.stem == j.stem:
    #         print( f"{i.stem} in out_path")
    #         break
    #     else:
    #         # print(f"{i.stem} not in out_path")
    #         yml_str = vs_obj.yaml(indent=True, sort_keys=False)
    #         new_path = out_path / f'{i.stem}.yml'
    #         new_path.write_text(yml_str)

/Users/ehaas/Documents/FHIR/davinci-ecdx/input/resources-yaml/codesystem-cdex-temp-codes.yml
writing codesystem-cdex-temp-codes to /Users/ehaas/Documents/FHIR/davinci-ecdx/input/resources-yaml/codesystem-cdex-temp-codes.yml


In [ ]:


fhir_test_server = 'http://test.fhir.org/r4'
fhir_test_server = 'http://hapi.fhir.org/baseR4'
fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1' 

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
     }
params = {}

# new_pou  = '''
# type:
#     coding:
#     - system: http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp
#       code: purpose-of-use
# valueCodeableConcept:
#     coding:
#     - system: http://terminology.hl7.org/CodeSystem/v3-ActReason
#       code: COVERAGE
# '''

new_pou  = '''
system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
code: operations-nos
'''

# print(new_pou)

# y_load(new_pou)

# new_tag  = '''
# system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
# code: claims-processing
# '''
new_tag  = '''
system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
code: authorization-other
'''

new_reasonref= '''
#  reference: http://example.org/fhir/Claim/cdex-example-claim
identifier: # ms 0..1 in data profile
    type:  # could add FILL=Payer or PLAC=Provider types
        coding:
            -  system: "http://terminology.hl7.org/CodeSystem/v2-0203"
               code: FILL 
               display: Filler Identifier
        text: Payer Claim ID 
    system: "http://example.org/cdex/payer/claim-ids"
    value: Claim123
'''
new_patientref= '''
#  reference: http://example.org/fhir/Patient/cdex-example-patient
identifier:
        type:
          coding:
            - system: "http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp" # vs http://hl7.org/fhir/us/carin-bb/CodeSystem/C4BBIdentifierType - um
              code: UMB
              display: Member Number
          text: Member Number
        system: "http://example.org/cdex/payer/member-ids"
        value: Member123
'''

new_payerref= '''
identifier:
    system: http://example.org/cdex/payer/payer-ids
    value: "Payer123"
'''
new_providerref= '''
identifier:
    system: http://hl7.org/fhir/sid/us-npi #NPI
    value: "9941339108"
'''

new_description = 'foo'

def convert(r):
    r = post(f'{fhir_test_server}/$convert', params = params, headers = headers, data = r)
    print(r.status_code)
    # view  output
    # return (r.json()["text"]["div"])
    return r.json()

def validate(r): #as dict
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    display(Markdown(f'### {r.status_code} {r.reason}\n{r.json()["text"]["div"]}')) 
    return r.json()

def write(data,f): #as dict 
    display(Markdown(f'### Writing to {f}....')) 
    #out= Path()/ "test_files" / "test.yml"
    out = Path(f)
    out.write_text(y_dump(data, sort_keys=False)) #as yaml

def entitle(name):
    x = name.split('-')
    return(' '.join(i.title() for i in x if i != "example"))

def enname(name):
    x = name.split('-')
    return(' '.join(i for i in x if i != "example"))

def p_name(f):
    #p = loads(f.open().read())
    p=f
    try:
        profile_url = (p['meta']['profile'][0])
        return profile_url.split('/')[-1]
    except KeyError:
        return p['resourceType'

In [ ]:

    data = y_load(i.read_text(),Loader=FullLoader) #dict
    print(i, data['resourceType'])
    if data['resourceType'] == 'Task':
        #print(f"old tag = { data['meta']['tag'][0]}")
        data['meta']['tag'][0]=y_load(new_tag,Loader=FullLoader)  # update work queue
        #print(f"new tag = { data['meta']['tag'][0]}")
        # update pou
        #print(f"old pou = { data['input'][1]['valueCodeableConcept']['coding'][0]}")
        data['input'][1]['valueCodeableConcept']['coding'][0]=y_load(new_pou,Loader=FullLoader)
        #print(f"new pou= { data['input'][1]['valueCodeableConcept']['coding'][0]}")
        # update reasonref
        #print(f"old reasonref = { data['reasonReference']}")
        data['reasonReference']=y_load(new_reasonref,Loader=FullLoader)
        #print(f"new reasonref= { data['reasonReference']}")
        # update reasonref
        #print(f"old patientref= { data['for']}")
        data['for']=y_load(new_patientref,Loader=FullLoader)
        #print(f"new patientref= { data['for']}")
        # update payerref
        #print(f"old payerref= { data['requester']}")
        data['requester']=y_load(new_payerref,Loader=FullLoader)
        #print(f"new payerref= { data['requester']}")
        # update providerref
        #print(f"old providerref= { data['owner']}")
        data['owner']=y_load(new_providerref,Loader=FullLoader)
        #print(f"new providerref= { data['owner']}")
        #print(y_dump(data, sort_keys=False))
        # validate(data)
        # data['meta']['extension'][1]['valueMarkdown'] += ' This example illustrates the use of business identifiers (instead of pointing to a FHIR resource) to references the various actors.' 
        # print(y_dump(data, sort_keys=False)) 
        # break
        write(data,i)